In [1]:
import os
import json
import vtk 
import slicer   
import MultiVolumeImporter  
import ExtractCenterline 

############################################################################
############################################################################

### SETUP/IMPORT ### 

#===========================================================================================
# CHANGE FILE PATHS, SEGMENT NAMES, PREPROCESSING VALUES AND OTHER MODIFIERS AS NEEDED
#===========================================================================================

print("Loading file for centerline extraction...")

# LOAD FILE AS VOLUME (CURRENTLY SAME AS SEGMENTATION) 
volume = slicer.util.loadVolume("J:\\SOHEIL_PROJECT\\SEGMENTATION\\MESAL-7012705\\Segmentation_final_v2-5_use-label.nrrd")

# LOAD IN A SEGMENTATION FILE 
# USED TO SPECIFY THAT FUTURE PROCESSES WILL INTERACT WITH THIS FILE (NOTE FUTURE USES OF 'vessel_seg')
vessel_seg = slicer.util.loadSegmentation("J:\\SOHEIL_PROJECT\\SEGMENTATION\\MESAL-7012705\\Segmentation_final_v2-5_use-label.nrrd")

segmentation = vessel_seg.GetSegmentation() 

# OPTION TO APPLY MODIFIER TO LOADED FILE (SMOOTHING EXAMPLE) 
segmentation.SetConversionParameter("Smoothing factor", "0.1") 

# OPTION TO CREATE THE 3D VIEW 
vessel_seg.CreateClosedSurfaceRepresentation() 

# ASSIGNS SPECIFIC SEGMENTATION LABEL ('Segment_1') TO VARIABLE 
segmentID = segmentation.GetSegmentIdBySegmentName('Segment_1') 

# CREATES AN EMPTY 'vtkPolyData' OBJECT 
segmentVtkPolyData = vtk.vtkPolyData() 

# GET 'vtkPolyData' REPRESENTATION OF SEGMENT SURFACE INTO 'segmentVtkPolyData' VARIABLE 
    # BY PASSING IN SEGMENTATION LABEL & 'vtkPolyData' VARIABLES 
vessel_seg.GetClosedSurfaceRepresentation(segmentID, segmentVtkPolyData) 

print(segmentID + " loaded for centerline extraction...") 


Loading file for centerline extraction...
Segment_1 loaded for centerline extraction...


In [2]:
############################################################################
############################################################################

### PREPROCESSING ### 
# USED TO SHORTN PATHS TO FUNCTIONS USED IN MODULE 
extractLogic = ExtractCenterline.ExtractCenterlineLogic() 

## PREPROCESS SEGMENTATION LABLEL ('Segment_1' IN THIS CASE) 
# USED TO INITIALIZE 'EXTRACT CENTERLINE' PARAMETERS 
inputSurfacePolyData = extractLogic.polyDataFromNode(vessel_seg, segmentID) 

# CAN BE MODIFIED AS NEEDED 
# SEE 'ExtractCenterline.py' FOR DEFAULT SETTINGS 
preprocessEnabled = True  # (self._parameterNode.GetParameter("PreprocessInputSurface") == "true") 
targetNumberOfPoints = 15000.0  # float(self._parameterNode.GetParameter("TargetNumberOfPoints")) 
decimationAggressiveness = 4.0 #float(self._parameterNode.GetParameter("DecimationAggressiveness")) 
subdivideInputSurface = False  # (self._parameterNode.GetParameter("SubdivideInputSurface") == "true") 
autoDetectEndPointsPushButton = True # (self._parameterNode.GetParameter(autoDetectEndPointsPushButton") == "true") 

slicer.util.showStatusMessage( 
    "Preprocessing surface before centerline extraction..." 
) 

print("Preprocessing surface before centerline extraction...") 

slicer.app.processEvents() 

# PASSESES IN THE ABOVE PREPROCESS VALUES 
# WILL BE USED DURING CENTERLINE EXTRACTION 
preprocessedPolyData = extractLogic.preprocess( 
    inputSurfacePolyData, 
    targetNumberOfPoints, 
    decimationAggressiveness, 
    subdivideInputSurface, 
) 
  
print("Preprocessing complete...")

Preprocessing surface before centerline extraction...
Preprocessing complete...


In [21]:
############################################################################
############################################################################

### EXTRACT CENTERLINE ### 

centerlineCurveNode = None 

print("Auto-detecting surface model endpoints...") 

# ATTEMPTS TO 'CLICK' THE 'AUTO-DETECT' BUTTON AND STORE THE END POINT MARKUPS IN THE 'endPointsMarkupsNode' VARIABLE 
# Auto-detect the endpoints 
endPointsMarkupsNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode", "Centerline endpoints") 
networkPolyData = extractLogic.extractNetwork(preprocessedPolyData, endPointsMarkupsNode) 
startPointPosition=None 
endpointPositions = extractLogic.getEndPoints(networkPolyData, startPointPosition) 
endPointsMarkupsNode.RemoveAllMarkups() 

for position in endpointPositions: 
  endPointsMarkupsNode.AddControlPoint(vtk.vtkVector3d(position)) 

print("End points autodetected...")

# USED TO STORE THE CENTERLINE CURVE
if centerlineCurveNode is None: 
    centerlineCurveNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsCurveNode", "Centerline curve") 

slicer.util.showStatusMessage("Extracting centerline...")  

# FORCES UPDATE TO ENSURE MODIFIED SETTINGS ARE APPLIED BEFORE RUNNING THE MODULE 
slicer.app.processEvents()  # force update 

# PASSES CENTERLINE/DIAGRAM DATA TO 2 SEPARATE VARIABLES 
# 'preprocessedPolyData'PREPROCESS MODIFIERS, MARKUPS/ENDPOINTS NODE PASSED INTO 'extractCenterline' 
centerlinePolyData, voronoiDiagramPolyData = extractLogic.extractCenterline( 
        preprocessedPolyData, endPointsMarkupsNode, curveSamplingDistance=0.001) #'end_pts' replaced with 'endPointsMarkupsNode' 

slicer.util.showStatusMessage("Generating curves and quantification results table...") 

slicer.app.processEvents()  # force update  

centerlinePropertiesTableNode = None  

print("Extracting centerline curve tree...")

# USED TO GENERATE CENTERLINE SEGMENTS FROM DATA
extractLogic.createCurveTreeFromCenterline( 
    centerlinePolyData, centerlineCurveNode, centerlinePropertiesTableNode 
)

if centerlineCurveNode.GetNumberOfControlPoints() == 2: 
    # If Extraction had an error, likely due to too high decimation aggressiveness 
    # Try again 
    slicer.util.errorDisplay( 
        "Centerline generation failed, possibly due to decimationAggressiveness being too high." 
    )

print("Segmented centerline curve tree created...")

Auto-detecting surface model endpoints...
End points autodetected...
Extracting centerline curve tree...


<class 'AttributeError'>: 'MRMLCorePython.vtkMRMLModelNode' object has no attribute 'SetControlPointPositionsWorld'

In [4]:
############################################################################
############################################################################

### GET 1-SEGMENT CENTERLINE MODEL ###

print("Extracting centerline model...")

# INSTANTIATE MODEL NODE
centerlineModel = slicer.vtkMRMLModelNode()
# APPLY 'centerlinePolyData'TO NEW MODEL NODE
centerlineModel.SetAndObservePolyData(centerlinePolyData)
# USED TO EDIT CENTERLINE MODEL APPEARANCE
centerlineModelDisplay = slicer.vtkMRMLModelDisplayNode()

# USED TO SET APPEARANCE OF MODEL NODE
centerlineModelDisplay.SetColor(1, 1, 0)
centerlineModelDisplay.BackfaceCullingOff()
centerlineModelDisplay.SetOpacity(0.8)
centerlineModelDisplay.SetPointSize(3)

centerlineModelDisplay.SetSliceIntersectionVisibility(True)
centerlineModelDisplay.SetVisibility(True)

# ADDS MODEL NODE APPEARANCE INFO TO SLICER SCENE
slicer.mrmlScene.AddNode(centerlineModelDisplay)
centerlineModel.SetAndObserveDisplayNodeID(centerlineModelDisplay.GetID())
centerlineModelDisplay.SetInputPolyDataConnection(centerlineModel.GetPolyDataConnection())
# ADDS CENTERLINE MODEL NODE TO SLICER SCENE
slicer.mrmlScene.AddNode(centerlineModel)

print("Centerline model extracted")

print("Centerline extraction complete")

Extracting centerline model...
Centerline model extracted
Centerline extraction complete


In [5]:
############################################################################
############################################################################

### EXPORT RESULTS TO FILES ###

print("Exporting 1-segment centerline model to file...")

# EXPORT 1-SEGMENT CENTERLINE MODEL TO .vtk FILE
slicer.util.saveNode(centerlineModel, "J:\\SOHEIL_PROJECT\\SEGMENTATION\\MESAL-7012705\\Segmentation_final_v2-5_use-label\\complete_centerline_model.vtk")

print("1-segment centerline model exported...")

print("Exporting centerline curvature segments to file...")

True